In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import json

# Heroes

### Heroes info

In [39]:
heroes_info = pd.read_csv('csv/heroes_done.csv').set_index('Unnamed: 0')

In [16]:
heroes_info.columns

Index(['1_pick', '1_win', '2_pick', '2_win', '3_pick', '3_win', '4_pick',
       '4_win', '5_pick', '5_win', '6_pick', '6_win', '7_pick', '7_win',
       '8_pick', '8_win', 'agi_gain', 'attack_range', 'attack_rate',
       'attack_type', 'base_agi', 'base_armor', 'base_attack_max',
       'base_attack_min', 'base_health', 'base_health_regen', 'base_int',
       'base_mana', 'base_mana_regen', 'base_mr', 'base_str', 'int_gain',
       'legs', 'move_speed', 'primary_attr', 'pro_ban', 'pro_pick', 'pro_win',
       'projectile_speed', 'str_gain', 'turn_rate', 'hero_magic_rate',
       'hero_picks', 'match_played', 'pick_rate', 'win_rate', 'kda_ratio',
       'gold_per_min', 'hero_damage_per_min', 'hero_healing_per_min',
       'kills_per_min', 'last_hits_per_min', 'lhten', 'stuns_per_min',
       'tower_damage', 'xp_per_min', 'carry_role', 'support_role',
       'pusher_role', 'jungler_role', 'durable_role', 'nuker_role',
       'initiator_role', 'disabler_role', 'escape_role'],
      dtyp

# Train

In [17]:
train = pd.read_csv('csv/train.csv').set_index('id')

In [18]:
heroes_train = pd.DataFrame([heroes_info.loc[hero] for hero in train['hero_id']])

In [19]:
heroes_train.index = train.index

In [20]:
train = pd.concat([train, heroes_train], axis=1)

In [21]:
train.head()

,skilled,player_team,winner_team,duration,pre_game_duration,first_blood_time,first_blood_claimed,hero_id,hero_pick_order,leaver_status,...,xp_per_min,carry_role,support_role,pusher_role,jungler_role,durable_role,nuker_role,initiator_role,disabler_role,escape_role
id,,,,,,,,,,,,,,,,,,,,,
7,1,dire,dire,2140,90,129,0,90,9,0,...,759.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
8,0,radiant,radiant,2138,90,174,0,5,5,0,...,702.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
12,0,radiant,radiant,3547,90,360,0,81,7,0,...,840.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
13,1,dire,radiant,1878,90,28,0,74,9,0,...,878.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
14,1,dire,radiant,2232,90,129,0,14,6,0,...,833.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0


In [22]:
dict(train.loc[7])

{'skilled': 1,
 'player_team': 'dire',
 'winner_team': 'dire',
 'duration': 2140,
 'pre_game_duration': 90,
 'first_blood_time': 129,
 'first_blood_claimed': 0,
 'hero_id': 90,
 'hero_pick_order': 9,
 'leaver_status': 0,
 'party_players': 3,
 'kills': 3,
 'deaths': 8,
 'assists': 22,
 'denies': 1,
 'level': 18,
 'net_worth': 10819,
 'gold': 1119,
 'gold_spent': 11765,
 'last_hits': 76,
 'gold_per_min': gold_per_min    375
 gold_per_min    527
 Name: 7, dtype: object,
 'xp_per_min': xp_per_min    434
 xp_per_min    759
 Name: 7, dtype: object,
 'hero_damage': 13262,
 'tower_damage': tower_damage     873
 tower_damage    3139
 Name: 7, dtype: object,
 'hero_healing': 155,
 'scaled_hero_damage': 8878,
 'scaled_tower_damage': 490,
 'scaled_hero_healing': 168,
 'stuns': 10.106689,
 'team_fight_participation': 0.6944444000000001,
 'observer_wards_placed': 3,
 'sentry_wards_placed': 3,
 'creeps_stacked': 8,
 'camps_stacked': 3,
 'rune_pickups': 4,
 'tower_kills': 0,
 'roshan_kills': 0,
 'near

In [23]:
train['duration_in_mins'] = train['duration'] / 60

In [24]:
train['gold_lose'] = train['gold_earned'] - train['gold_spent']
train['gold_lose_ratio'] = train['gold_lose'] / train['gold_earned']

KeyError: 'gold_earned'

In [25]:
for feature in ['hero_damage', 'last_hits', 'stuns']:
    train[feature + '_per_min'] = train[feature] / train['duration_in_mins']

In [26]:
train['avg_kda_x16'] = (train['avg_kills_x16'] + train['avg_assists_x16']) / train['avg_deaths_x16']
train['kda'] = (train['kills'] + train['assists']) / train['deaths']

In [27]:
list(train.columns)

['skilled',
 'player_team',
 'winner_team',
 'duration',
 'pre_game_duration',
 'first_blood_time',
 'first_blood_claimed',
 'hero_id',
 'hero_pick_order',
 'leaver_status',
 'party_players',
 'kills',
 'deaths',
 'assists',
 'denies',
 'level',
 'net_worth',
 'gold',
 'gold_spent',
 'last_hits',
 'gold_per_min',
 'xp_per_min',
 'hero_damage',
 'tower_damage',
 'hero_healing',
 'scaled_hero_damage',
 'scaled_tower_damage',
 'scaled_hero_healing',
 'stuns',
 'team_fight_participation',
 'observer_wards_placed',
 'sentry_wards_placed',
 'creeps_stacked',
 'camps_stacked',
 'rune_pickups',
 'tower_kills',
 'roshan_kills',
 'nearby_creep_death_count',
 'radiant_tower_status',
 'dire_tower_status',
 'radiant_barracks_status',
 'dire_barracks_status',
 'fight_score',
 'farm_score',
 'support_score',
 'push_score',
 'avg_kills_x16',
 'avg_deaths_x16',
 'avg_assists_x16',
 'avg_gpm_x16',
 'avg_xpm_x16',
 'best_kills_x16',
 'best_assists_x16',
 'best_gpm_x16',
 'best_xpm_x16',
 'win_streak',
 '

In [28]:
for par in ['str', 'agi', 'int']:
    train[par] = train['base_' + par] + train[par + '_gain'] + train['level']

In [29]:
for par in [('health', 'str', 20), ('health_regen', 'str', 0.1), ('armor', 'agi', 0.16), ('mana', 'int', 12), ('mana_regen', 'int', 0.05)]:
    train[par[0]] = train['base_' + par[0]] + train[par[1]] * par[2]

# Json Lines

In [31]:
def mmmm(table, result):
    for feature in table.keys():
        result[feature + '_mean'] = np.mean(table[feature])
        result[feature + '_median'] = np.median(table[feature])
        result[feature + '_max'] = np.max(table[feature])
        result[feature + '_min'] = np.min(table[feature])

### Items

In [49]:
items_info = pd.read_csv('csv/dota2_items.csv')

def items_sum(record):
    result = {('item_%d' % ids): 0 for ids in items_info['item_id']}
    for item in record['item_purchase_log']:
        result['item_%d' % item['item_id']] += 1
    return result

### Abilities

In [33]:
import ast
abilities_info = pd.read_csv('csv/dota2_abilities.csv')

get_behaviors = {ids: ast.literal_eval(bhs.replace('nan', '\'nan\'')) for ids, bhs in zip(abilities_info['ability_id'], abilities_info['behavior'])}
behaviors = {bh for bhs in abilities_info['behavior'] for bh in ast.literal_eval(bhs.replace('nan', '\'nan\''))}

def abilities(record):
    result = {bh: 0 for bh in behaviors}
    for ids in record['ability_upgrades']:
        for bh in get_behaviors[ids]:
            result[bh] += 1
    return result

### Kill log

In [34]:
def kill_log(record):
    damage_targets = record['damage_targets']
    result = {}
    sorted_targets = {}
    for target in damage_targets.keys():
        kind = target.split('_')[2]
        if kind in ['observer', 'sentry']:
            result[kind + '_ward_damage'] = damage_targets[target]
        else:
            if kind in ['neutral', 'hero']:
                pass
            elif kind in ['creep']:
                if target.split('_')[3] == 'goodguys':
                    kind = 'friendly_creeps' if record['player_team'] == 'dire' else 'enemy_creeps'
                else:
                    kind = 'friendly_creeps' if record['player_team'] == 'radiant' else 'enemy_creeps'
            else:
                kind = 'hero_minions'
            kind += '_damage'
            targets = sorted_targets.get(kind, [])
            targets.append(damage_targets[target])
            sorted_targets[kind] = targets

    mmmm(sorted_targets, result)
    
    return result


### Gold log

In [35]:
def gold_log(record):
            
    series = record['series']

    drop_count = sum([1 for s in list(map(list, zip(record['series']['time'], record['series']['player_gold'], record['series']['dire_gold'], record['series']['radiant_gold']))) if 0 in s or s[0] < 0])
    for p in series.keys(): series[p] = series[p][drop_count:]

    radiant = 'friendly_gold' if record['player_team'] == 'radiant' else 'enemy_gold'
    dire = 'friendly_gold' if record['player_team'] == 'dire' else 'enemy_gold'

    series[radiant] = series.pop('radiant_gold')
    series[dire] = series.pop('dire_gold')

    for p in ['player_gold', 'friendly_gold', 'enemy_gold']:
        for t in range(len(series['time']) - 1):
            arr = series.get(p + '_growth', [])
            arr.append(series[p][t + 1] - series[p][t])
            series[p + '_growth'] = arr
        series[p + '_growth'] += [np.mean(series[p + '_growth'])]

    for p in ['friendly_gold', 'enemy_gold']:
        series[p + '_ratio'] = np.array(series['player_gold']) / np.array(series[p])
        series[p + '_growth_ratio'] = np.array(series['player_gold_growth']) / np.array(series[p + '_growth'])

    series['teams_gold_ratio'] = np.array(series['friendly_gold']) / np.array(series['enemy_gold'])

    result = {}

    result['gold_earned'] = series['player_gold'][-1]
    result['gold_after_warmup'] = series['player_gold'][0]

    series = {k: series[k] for k in list(series.keys())[4:]}

    mmmm(series, result)
    
    return result

### Other heroes

In [36]:
def other_heroes(record):
    friendly_heroes = record['radiant_heroes'] if record['player_team'] == 'radiant' else record['dire_heroes']
    enemy_heroes = record['radiant_heroes'] if record['player_team'] != 'radiant' else record['dire_heroes']

    hero_ids = list(heroes_info.index)

    result = {}

    for hero_id in hero_ids:
        if hero_id in friendly_heroes:
            result[hero_id] = 1
        elif hero_id in enemy_heroes:
            result[hero_id] = -1
        else:
            result[hero_id] = 0
    return result

### Level Up Times

In [37]:
def level_up_times(record):
    level_up_times = record['level_up_times']
    level_up_times_diff = {'level_up_times': [level_up_times[p + 1] - level_up_times[p] for p in range(len(level_up_times) - 1)]} if len(level_up_times) > 1 else {'level_up_times': [0]}
    result = {}
    mmmm(level_up_times_diff, result)
    return result

### Filling

In [40]:
damage_targets, teams_heroes, serieses, level_times, items_count, abilities = {}, {}, {}, {}

with open('jsonlines/train.jsonlines') as file:
    for i, line in enumerate(file):
        record = json.loads(line)
        damage_targets[record['id']] = kill_log(record)
        teams_heroes[record['id']] = other_heroes(record)
        serieses[record['id']] = gold_log(record)
        level_times[record['id']] = level_up_times(record)
        items_count[record['id']] = items_sum(record)
        abilities[record['id']] = abilities(record)
        if i % 10 == 0:
            print('\r{}'.format(i), end='')

470

KeyboardInterrupt: 

In [36]:
serieses = pd.DataFrame(serieses).T

In [37]:
teams_heroes = pd.DataFrame(teams_heroes).T

In [62]:
level_times = pd.

{7: {'level_up_times_mean': 126.375,
  'level_up_times_median': 134.0,
  'level_up_times_max': 210,
  'level_up_times_min': 32},
 8: {'level_up_times_mean': 105.05263157894737,
  'level_up_times_median': 90.0,
  'level_up_times_max': 185,
  'level_up_times_min': 6},
 12: {'level_up_times_mean': 99.65217391304348,
  'level_up_times_median': 102.0,
  'level_up_times_max': 193,
  'level_up_times_min': 37},
 13: {'level_up_times_mean': 100.85714285714286,
  'level_up_times_median': 103.0,
  'level_up_times_max': 179,
  'level_up_times_min': 21},
 14: {'level_up_times_mean': 138.33333333333334,
  'level_up_times_median': 120.0,
  'level_up_times_max': 388,
  'level_up_times_min': 37},
 15: {'level_up_times_mean': 112.21739130434783,
  'level_up_times_median': 92.0,
  'level_up_times_max': 268,
  'level_up_times_min': 18},
 19: {'level_up_times_mean': 103.5,
  'level_up_times_median': 104.0,
  'level_up_times_max': 297,
  'level_up_times_min': 5},
 23: {'level_up_times_mean': 97.222222222222

In [38]:
damage_targets

{7: {'sentry_ward_damage': 8,
  'observer_ward_damage': 8,
  'friendly_creeps_damage_mean': 17179.0,
  'friendly_creeps_damage_median': 17179.0,
  'friendly_creeps_damage_max': 25044,
  'friendly_creeps_damage_min': 9314,
  'neutral_damage_mean': 1190.5,
  'neutral_damage_median': 1125.0,
  'neutral_damage_max': 2376,
  'neutral_damage_min': 51,
  'hero_minions_damage_mean': 198.30769230769232,
  'hero_minions_damage_median': 93.0,
  'hero_minions_damage_max': 820,
  'hero_minions_damage_min': 19,
  'hero_damage_mean': 4764.6,
  'hero_damage_median': 3009.0,
  'hero_damage_max': 13548,
  'hero_damage_min': 1598,
  'enemy_creeps_damage_mean': 73.0,
  'enemy_creeps_damage_median': 73.0,
  'enemy_creeps_damage_max': 103,
  'enemy_creeps_damage_min': 43}}

In [39]:
train = pd.concat([serieses, teams_heroes, damage_targets], axis=1)

TypeError: cannot concatenate object of type "<class 'dict'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid

In [40]:
train.head()

,skilled,player_team,winner_team,duration,pre_game_duration,first_blood_time,first_blood_claimed,hero_id,hero_pick_order,leaver_status,...,avg_kda_x16,kda,str,agi,int,health,health_regen,armor,mana,mana_regen
id,,,,,,,,,,,,,,,,,,,,,
7,1,dire,dire,2140,90,129,0,90,9,0,...,2.125000,3.125000,36.3,34.6,44.2,926.0,3.63,4.536,605.4,2.210
8,0,radiant,radiant,2138,90,174,0,5,5,0,...,2.555556,2.888889,41.2,38.6,38.3,1024.0,4.12,6.176,534.6,1.915
12,0,radiant,radiant,3547,90,360,0,81,7,0,...,3.142857,6.200000,50.7,41.1,44.2,1214.0,5.07,7.576,605.4,2.210
13,1,dire,radiant,1878,90,28,0,74,9,0,...,3.142857,1.333333,36.4,31.9,34.6,928.0,3.64,4.104,490.2,1.730
14,1,dire,radiant,2232,90,129,0,14,6,0,...,2.400000,1.300000,46.0,32.5,32.5,1120.0,4.60,3.200,465.0,1.625


In [ ]:
pd.DataFrame(serieses).head()

In [ ]:
pd.DataFrame(teams_heroes).head()

In [83]:
items_info = pd.read_csv('csv/dota2_items.csv')

In [84]:
items = {('final_item_%d' % ids): 0 for ids in items_info['item_id']}

In [87]:
final_items = {}
with open('jsonlines/train.jsonlines') as file:
    for i, line in enumerate(file):
        record = json.loads(line)
        result = items.copy()
        for item in record['final_items']:
            if item != 0:
                result[('final_item_%d' % item)] += 1
        final_items[record['id']] = result
        if i % 10 == 0:
            print('\r{}'.format(i), end='')
            
with open('jsonlines/test.jsonlines') as file:
    for i, line in enumerate(file):
        record = json.loads(line)
        result = items.copy()
        for item in record['final_items']:
            if item != 0:
                result[('final_item_%d' % item)] += 1
        final_items[record['id']] = result
        if i % 10 == 0:
            print('\r{}'.format(i), end='')

43260

In [89]:
pd.DataFrame(final_items).T.to_csv('final_items.csv')